In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from google.colab import files

# ⚡ 1️⃣ دریافت و استخراج دیتاست و تست
uploaded = files.upload()
!unrar x bump_on.rar
!unrar x test.rar

dataset_path = "bump_on"
test_path = "test"

# 📌 نمایش محتوای فولدرها
print("📂 محتوای bump_on:")
!ls -l bump_on

print("\n📂 محتوای test:")
!ls -l test

# 🔹 2️⃣ خواندن کلاس‌ها از classes.txt
classes_file = os.path.join(dataset_path, "classes.txt")
with open(classes_file, "r") as f:
    class_names = [line.strip() for line in f.readlines()]
num_classes = len(class_names)

print(f"\n🔹 کلاس‌های دیتاست: {class_names}")

# 📏 اندازه تصاویر
img_size = (224, 224)

#  3️⃣ تابع بارگذاری داده‌ها (اصلاح شده برای تست بدون لیبل)
def load_images(folder, labeled=True):
    images = []
    labels = []
    for file in os.listdir(folder):
        if file.endswith(".jpg"):
            img_path = os.path.join(folder, file)

            # خواندن تصویر
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img is None:
                print(f"⚠️ تصویر بارگذاری نشد: {img_path}")
                continue

            img = cv2.resize(img, img_size) / 255.0
            images.append(img)

            # اگر داده‌ها دارای لیبل هستند، لیبل را بخواند
            if labeled:
                txt_path = img_path.replace(".jpg", ".txt")
                if os.path.exists(txt_path):
                    with open(txt_path, "r") as f:
                        try:
                            class_id = int(f.readline().split()[0])
                            labels.append(class_id)
                        except:
                            print(f"⚠️ مشکل در خواندن کلاس از {txt_path}")
                            labels.append(0)  # مقدار پیش‌فرض
                else:
                    print(f"⚠️ فایل کلاس وجود ندارد: {txt_path}")
                    labels.append(0)  # مقدار پیش‌فرض

    print(f"📊 تعداد تصاویر پردازش‌شده از {folder}: {len(images)}")

    if labeled:
        return np.array(images), np.array(labels)
    else:
        return np.array(images)  # فقط تصاویر برگرداند

#  4️⃣ بارگذاری مجموعه داده‌ها
X_train, y_train = load_images(dataset_path, labeled=True)
X_test = load_images(test_path, labeled=False)

print(f"\n✅ تعداد تصاویر آموزشی: {len(X_train)}, تعداد تصاویر تست: {len(X_test)}")

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation="relu")(x)
x = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=5, validation_split=0.1)

predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)
plt.figure(figsize=(10, 5))
for i in range(min(5, len(X_test))):
    plt.subplot(1, 5, i + 1)
    plt.imshow(X_test[i])
    plt.title(f"پیش‌بینی: {class_names[predicted_labels[i]]}")
    plt.axis("off")
plt.show()